## Getting Polyline data from Studio and transferring to Geodatabase

With recent updates to Studio database the geometry type that is stored within the database was changed and ArcGIS products were no longer able to read it. There were work-arounds involving creating copies of query layers or other creative methods that added steps onto what use to be a strightforward method.


This notebook suggests one work around that leverages the geometry options within the arcgis library.
The basic run down: 

1. Write a query to the Studio database grabbing a table that has a shape field for a polyline
2. Pull out geometry information from SDO_object contained in shape field
3. Feed the geometry information to the Polyline function in arcgis library. This will make a true polyline geometry
4. Make every row into a Feature containing a dictionary of every row containing the attributes as well as the geometry 
5. Make a FeatureSet from a list of all the Features
6. Turn into spatial dataframe and save to file 

Originally created 1/22/2021  
---Noah Karsky---


In [4]:
'''###IMPORT  LIBRARIES#####

from arcgis.geometry import Point, Polyline, Polygon, Geometry
import pandas as pd
import arcgis
from arcgis.features import SpatialDataFrame as sdf
from arcgis.features import Feature, FeatureSet, FeatureCollection


# these libraries would have to be downloaded from Nexus as I dont think they ship with ArcPro 2.5
import cx_Oracle
import sqlalchemy
from sqlalchemy import create_engine
import os
import arcpy
'''


In [5]:
QUERY = '''SELECT UWI,
       BOREHOLE_ID,
       BOREHOLENAME,
       BULKARRAYID,
       ID,
       ISACTIVE,
       GUID,
       EXTENDEDPROPERTIES_,
       INSERT_DATE,
       LINE_,
       NAME,
       MD,
       TVD,
       FOLDER_PATH,
       SHAPE,
       UPDATE_DATE
  FROM (SELECT ROOT.BOREHOLE_ID BOREHOLE_ID,
               BOREHOLE.NAME BOREHOLENAME,
               ROOT.DEVIATION_SURVEY_BULK_DATA_ID BULKARRAYID,
               ROOT.REMARKS COMMENTS,
               ROOT.COORDINATE_SYSTEM_ID COORDINATESYSTEM_ID,
               CRS CRSCODE,
               EXTENDEDPROPERTIES.ID EXTENDEDPROPERTIES_,
               ROOT.GUID GUID,
               ROOT.ID ID,
               BOREHOLE.DRILLER_TD_MD MD,
               BOREHOLE.DRILLER_TD_TVD TVD,
               ROOT.INSERT_DATE INSERT_DATE,
               ROOT.INSERT_USER INSERT_USER,
               CASE
                  WHEN    DEFINITIVE_SURVEY_ID = ROOT.ID
                       OR ACTIVE_PLAN_ID = ROOT.ID
                  THEN
                     1
                  ELSE
                     0
               END
                  ISACTIVE,
               LINE.ID LINE_,
               ROOT.NAME NAME,
               ORIGINAL_COORD_SYSTEM_ID ORIGINALCOORDSYSTEM_ID,
               LINE.SHAPE SHAPE,
               PARENT_GUID PARENTGUID,
               CASE Root.Existence_Kind
                  WHEN 'Explicit' THEN 'Survey'
                  WHEN 'XYZ' THEN 'Survey'
                  WHEN 'MdIncAzi' THEN 'Survey'
                  WHEN 'XYTvd' THEN 'Survey'
                  WHEN 'DxDyTvd' THEN 'Survey'
                  WHEN 'AutoPlan' THEN 'Plan'
                  WHEN 'ExplicitPlan' THEN 'Plan'
                  WHEN 'XYZPlan' THEN 'Plan'
               END
                  SURVEYTYPE,
               ROOT.TIE_IN_MD TIEINMD,
               ROOT.PARENT_SURVEY_ID TIEINSURVEYPLAN_ID,
               PARENTSURVEYPLAN.NAME TIEINSURVEYPLANNAME,
               ROOT.INTERPRETATION_STATUS TRAJECTORYFORMAT,
               PROPERTIES TRAJECTORYPROPERTIES,
               ROOT.UPDATE_DATE UPDATE_DATE,
               ROOT.UPDATE_USER UPDATE_USER,
               SELF.Folder_Path,
               UBHI UWI
          FROM SAK01.DEVIATION_SURVEY_ ROOT
               INNER JOIN SAK01.COORDINATE_SYSTEM_ COORDINATESYSTEM
                  ON (ROOT.COORDINATE_SYSTEM_ID = COORDINATESYSTEM.ID)
               LEFT OUTER JOIN SAK01.DEVIATION_SURVEY_ PARENTSURVEYPLAN
                  ON (ROOT.PARENT_SURVEY_ID = PARENTSURVEYPLAN.ID)
               LEFT OUTER JOIN SAK01.DBX_OBJECT_ SELF
                  ON (SELF.ENTITY_ID = ROOT.ID)
               LEFT OUTER JOIN SAK01.LINE_ LINE
                  ON (LINE.ID = ROOT.SURVEY_TRAJECTORY_ID)
               LEFT OUTER JOIN
               SAK01.DBX_EXTENDED_PROPERTIES_ EXTENDEDPROPERTIES
                  ON (EXTENDEDPROPERTIES.ENTITY_ID = ROOT.ID)
               INNER JOIN SAK01.BOREHOLE_ BOREHOLE
                  ON (BOREHOLE.ID = ROOT.BOREHOLE_ID)) MAIN_VIEW
                  WHERE ISACTIVE=1 AND LINE_ IS NOT NULL
'''



In [6]:
##Making the engine that connects to Studio database
engine = create_engine('oracle://@P_PET_SAMP') # creating a connection to the studio database
connection = engine.connect()
print(engine)



Engine(oracle://@P_PET_SAMP)


In [4]:
##this first function was puled from  a stack overflow thread it allows us to pull apart the oracle obj

def ObjectRepr(obj):
    if obj.type.iscollection:
        returnValue = []
        for value in obj.aslist():
            if isinstance(value, cx_Oracle.Object):
                value = ObjectRepr(value)
            returnValue.append(value)
    else:
        returnValue = {}
        for attr in obj.type.attributes:
            value = getattr(obj, attr.name)
            if value is None:
                continue
            elif isinstance(value, cx_Oracle.Object):
                value = ObjectRepr(value)
            returnValue[attr.name] = value
    return returnValue


#the part we care about inside of the oracle object is the SDO_ORDINATES and in our example it has 4 parts to each point
#this function takes the big list of all the points and breaks it into many lists with 4 elements
def divide_chunks(l, n): 
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 

#This function applies the above two functions and then removes the 4th element in every point as it is "none"
#then it forms a dictionary called line that has every point
#it then feeds the dictionary to the Polyline function
def make_geom(shape_item,):
    get_obj = ObjectRepr(shape_item)
    path_list = list(divide_chunks(get_obj['SDO_ORDINATES'],4))      #grabs SDO_ORDINATE and cunks it into lists of 4
    [x.pop() for x in path_list]         #remove 4th element from every point leaving x,y,z
    #builds a line dictionary using the path_list and assumes WGS84
    line = {
    "paths": [path_list],
    "spatialReference" : {"wkid" : 4326}}
    polyline = Polyline(line,hasZ=True)#feeds the line to Polyline making polyline function
    return polyline


def make_featureclass(dataframe):
    dataframe.dropna(subset=['shape'],inplace=True) #we drop rows with no geometry
    dataframe['SHAPE'] = dataframe['shape'].apply(lambda x: make_geom(x)) #takes prior function and applies it every row
    dataframe.drop('shape',axis=1,inplace=True)   #we no longer need the old shape
    inx_array = dataframe.index.values 
    feature_list = []
    for x in inx_array:
        ent1 = dataframe.iloc[x,-1]  #the attribute data for each row
        ent2 = dataframe.iloc[x,:-1].to_dict()    #the geometry data for each row
        feature = Feature(geometry=ent1,attributes=ent2) #making a feature from each row
        feature_list.append(feature)   #making a featureSet from each row
    return FeatureSet(features=feature_list)





In [5]:
df = pd.read_sql_query(sql = QUERY,con = connection) 

In the cell above if you scroll to the end you can see the Shape column. it is stored as a special type of cx_Oracle object

In [6]:
print(' this is what the Oracle object looks like: {} so we need a way to unpack it'.format(df.loc[45,'shape']))

 this is what the Oracle object looks like: <cx_Oracle.Object MDSYS.SDO_GEOMETRY at 0x3a286490> so we need a way to unpack it


In [7]:
#use ObjRepair function on it
ObjectRepr(df.loc[45,'shape'])

{'SDO_GTYPE': 4402,
 'SDO_ELEM_INFO': [1, 2, 1],
 'SDO_ORDINATES': [143.285860899996,
  52.4766366100001,
  18.2,
  None,
  143.285860899996,
  52.4766368770145,
  5.40004598077211,
  None,
  143.285858793254,
  52.476638284751,
  -19.7789137714722,
  None,
  143.285855183726,
  52.4766399615372,
  -34.6856094779166,
  None,
  143.285850265809,
  52.4766424321388,
  -48.8089601089738,
  None,
  143.285844326947,
  52.4766457662463,
  -64.7195129498794,
  None,
  143.285841942825,
  52.4766473224444,
  -72.0856841763719,
  None,
  143.285840931658,
  52.4766480382447,
  -76.3743791139596,
  None,
  143.285839713399,
  52.4766486382833,
  -85.0436312866089,
  None,
  143.28583888498,
  52.4766488892202,
  -93.6833904969908,
  None,
  143.285838514564,
  52.4766491276476,
  -98.4732483820642,
  None,
  143.285838004845,
  52.4766494353496,
  -109.293132768948,
  None,
  143.285837620997,
  52.4766496025936,
  -119.633082667849,
  None,
  143.285837355443,
  52.4766496400584,
  -126.583054

The output above are all the elements that were inside of the Oracle object, They all correspond to data that within the SDO geometry. One can understand what all these mean with a simple websearch, but we are only after the SDO_ORDINATES.

We are essentially given all the different points within a polygon, and we need to break it up into each point. 

To do this, we split it into a list of 4 elements (x,y,z,m) ***I think it is m*** This feature class has no m, so we end up popping it off to leave us with an  x,y,z that we then format for use in the arcgis library.

That really details the part that took me forever to find out. The rest of the code consists of formatting these coordinates into dictionaries to be used by the arcgis library. If you want to look into the code more all the formatting happens in the functions above

In [8]:
df = pd.read_sql_query(sql = QUERY,con = connection) 
feature_set = make_featureclass(df)

In [9]:
feature_set

<FeatureSet> 533 features

At this point we have a feature set that has been loaded with all our data and we can turn it into a data frame

In [10]:
feature_sdf = feature_set.sdf

#to turn it into a feature class you would use the method below:
#export_df.spatial.to_featureclass(location=fc_location,has_z=True,has_m=True)


In [11]:
feature_sdf.boreholename.sort_values().unique()

array(['201 / 203', '202', '204', '204ST1', '205', '205ST1', '206',
       '206ST1', '207', '207PL', '208', '209', '209ST1', '210', '210ST1',
       '210ST2', '211', '212', '215', '215ST1', '215ST2', '216', '216ST1',
       '216ST2', '217', '217ST1', '217ST2', '218', '218PL', '218ST1',
       '219', '219ST1', '219ST2', '219ST3', '220', '220PL', '220ST1',
       '220ST2', '220ST3', '221', '222', '222PL', '222ST1', '223', '224',
       '226', '227', '228', '228PL', '228ST1', '229', '233', '233ST1',
       '234', '234ST1', '234ST2', '234ST3', '234ST4', '235', '235ST1',
       '240', '241', '242', '243', '244', '244PL', '246', '247', '255',
       '255PL', '256', '256PL', '256ST1', '256ST2', '277', '278', '279',
       'AD SWW-1', 'AD SWW-2', 'AD SWW-3', 'ARKUTUN-DAGI DD-1',
       'ARKUTUN-DAGI DP-10D', 'ARKUTUN-DAGI DP-11D',
       'ARKUTUN-DAGI DP-12D', 'ARKUTUN-DAGI DP-14L',
       'ARKUTUN-DAGI DP-14L_Surface Hole', 'ARKUTUN-DAGI DP-16D',
       'ARKUTUN-DAGI DP-17U', 'ARKUTUN-DAGI DP

### This is all part 2

In this portion we are pulling reservoir segments from EMSDB and joining it to the data frame. Also doing data qc and making the feature class more useful 

In [106]:

query_for_reservoirs = '''
 SELECT ID_COMPLETION_RESERVOIR,
       ID_COMPLETION,
       ZONE_ALIAS,
       PRIMARY_FLAG,
       COMMINGLED_FLAG,
       RESERVOIR_SEG_IDENTIFIER,
       RESERVOIR_SEG_NAME,
       RESERVOIR_NAME,
       RESERVOIR_IDENTIFIER,
       WELLBORE_IDENTIFIER,
       WELLBORE_NAME,
       WELLBORE_ALIAS,
       FIELD_NAME
  FROM EG.COMPLETION_RESERVOIR_MVIEW
  '''

engine2 = create_engine('oracle://@YUZSDB') # creating a connection to the studio database
connection2 = engine2.connect()
print(engine2)

Engine(oracle://@YUZSDB)


In [107]:
emsdb_df = pd.read_sql_query(sql = query_for_reservoirs,con = connection2) 
emsdb_df = emsdb_df[['wellbore_identifier', 'wellbore_name', 'field_name','commingled_flag','reservoir_seg_name']]
emsdb_df.wellbore_identifier = emsdb_df.wellbore_identifier.str.strip()
emsdb_df.set_index('wellbore_identifier',inplace=True)

# reservoir_dummies is a big ol pivot
reservoir_dummies = pd.get_dummies(emsdb_df.reservoir_seg_name,prefix = 'res').groupby('wellbore_identifier').sum()

emsdb_df.drop_duplicates(['wellbore_name'],inplace=True)
emsdb_df.drop(['reservoir_seg_name'],axis=1,inplace=True)
#join the emsdb stuff together
emsdb_join= pd.merge(emsdb_df,reservoir_dummies,left_index=True,right_index=True)


In [119]:
#join emsdb stuff to the original dataframe
export_df = pd.merge(feature_sdf.set_index('uwi'),emsdb_join,left_index=True,right_index=True,how='left').reset_index()
export_df.drop([ 'bulkarrayid','isactive','name', 'update_date', 'OBJECTID','id','borehole_id','line_','wellbore_name','extendedproperties_',],axis =1 ,inplace=True)

#rename columns
export_df.columns = ['uwi', 'boreholename', 'guid',  'insert_date',
       'md', 'tvd','folder_path', 'SHAPE', 'field_name', 'commingled_flag', 'res_Chayvo_14',
       'res_Chayvo_16', 'res_Chayvo_17', 'res_Chayvo_18', 'res_Chayvo_19',
       'res_Chayvo_20', 'res_Dagi_10-1', 'res_Dagi_10-2', 'res_Dagi_11-1', 'res_Dagi_11-2',
       'res_Dagi_12-1', 'res_Dagi_12-2', 'res_Dagi_13-1', 'res_Dagi_13-2',
       'res_Dagi_13-3', 'res_Dagi_14-1', 'res_Dagi_14-2',
       'res_Odoptu_21-1_LEB_TSA', 'res_Odoptu_21-1_OD_TSA', 'res_Odoptu_21-2',
       'res_Odoptu_21-3_4_North', 'res_Odoptu_22-1_North', 'res_Odoptu_22-2',
       'res_Odoptu_22-3']


#making a short well nameexport_df['well_short_name'] = export_df.boreholename
export_df['well_short_name'] = export_df['boreholename'].str.replace('ODOPTU ','').str.replace('ARKUTUN-DAGI ','').str.replace('CHAYVO ','')
#making a semi-useful folder there are some glitches which is why we need to catch them with extra code

export_df['folder'] = export_df.folder_path.str.split('\\').str[-2]
index = export_df.loc[export_df.folder_path.str.contains('Production')].index
export_df.loc[index,'folder'] = 'Production'

index_def = export_df.loc[export_df.folder_path.str.contains('def')].index
export_df.loc[index_def,'folder'] = 'Production'


index_bh = export_df.loc[export_df.boreholename.str.contains('ARKUTUN-DAGI DW-32D')].index
export_df.loc[index_bh,'folder'] = 'Production'



export_df['top_md'] = 0


#doing a bunch of string manipulation to help populate some of the field_name
export_df.loc[export_df[(export_df.field_name.isna())&(export_df.boreholename.str.contains('DW-|DP-|DAGI|ARKUTUN'))].index,'field_name'] ='Arkutun-Dagi'
export_df.loc[export_df[(export_df.field_name.isna())&(export_df.folder_path.str.contains('CSD'))].index,'field_name'] = 'Arkutun-Dagi'
export_df.loc[export_df[(export_df.field_name.isna())&(export_df.folder_path.str.contains('Odoptu'))].index,'field_name'] = 'Odoptu'
export_df.loc[export_df[(export_df.field_name.isna())&(export_df.boreholename.str.contains('ODOPTU'))].index,'field_name'] ='Odoptu'
export_df.loc[export_df[(export_df.field_name.isna())&(export_df.folder_path.str.contains('Chayvo'))].index,'field_name'] = 'Chayvo'
export_df.loc[export_df[(export_df.field_name.isna())&(export_df.boreholename.str.contains('CHAYVO'))].index,'field_name'] ='Chayvo'

In [120]:
check_df= export_df[export_df.field_name == 'Arkutun-Dagi'].sort_values('insert_date',ascending=False)
check_df = check_df[['boreholename','insert_date','res_Dagi_10-1', 'res_Dagi_10-2', 'res_Dagi_11-1', 'res_Dagi_11-2',
       'res_Dagi_12-1', 'res_Dagi_12-2', 'res_Dagi_13-1', 'res_Dagi_13-2',
       'res_Dagi_13-3', 'res_Dagi_14-1', 'res_Dagi_14-2','folder_path','folder']]
check_df.head(20)

,boreholename,insert_date,res_Dagi_10-1,res_Dagi_10-2,res_Dagi_11-1,res_Dagi_11-2,res_Dagi_12-1,res_Dagi_12-2,res_Dagi_13-1,res_Dagi_13-2,res_Dagi_13-3,res_Dagi_14-1,res_Dagi_14-2,folder_path,folder
526,DW-31U,2021-06-21 15:07:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\Project\Input\Wells\Working\DW-31U\Surveys an...,DW-31U
525,DW-38U,2021-06-21 15:07:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\Project\Input\Wells\Working\DW-38U\Surveys an...,DW-38U
324,ARKUTUN-DAGI DW-32D,2021-06-21 00:34:09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\Project\Input\Wells\UNKNOWN\ARKUTUN-DAGI DW-3...,Production
326,ARKUTUN-DAGI DW-23D,2021-05-10 18:16:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\Project\Input\Wells\Production\ARKUTUN-DAGI D...,Production
496,DW-41D,2021-04-13 08:37:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\Project\Input\Wells\Working\DW-41D\Surveys an...,DW-41D
322,ARKUTUN-DAGI DP-16D,2021-03-11 00:25:52,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,\Project\Input\Wells\Production\ARKUTUN-DAGI D...,Production
311,ARKUTUN-DAGI DP-35U,2021-02-08 17:08:28,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,\Project\Input\Wells\Production\ARKUTUN-DAGI D...,Production
317,ARKUTUN-DAGI DW-19D,2021-01-06 19:13:18,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,\Project\Input\Wells\Production\ARKUTUN-DAGI D...,Production
464,DP-27L,2020-11-17 13:09:38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\Project\Input\Wells\Working\DP-27L\Surveys an...,DP-27L
461,DP-20D-RD,2020-11-17 13:09:38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\Project\Input\Wells\Working\DP-20D-RD\Surveys...,DP-20D-RD


In [121]:
export_df.loc[export_df.folder_path.str.contains('Production'),'folder'].value_counts()

Production    218
Name: folder, dtype: int64

## Part 3 

We export our feature class and then create routes along it to get an M value. 

In [16]:
arcpy.env.workspace = r'D:\work_2021\January\IMPORT_TO_SDE.gdb'
arcpy.env.overwriteOutput = True

fc_location = r'I:\UTCS\Region\Russia\Assets\Sakhalin_1.Hou\Regional\Master\SubSrfc\Geo\ArcGis\Geol_Anal\Data\Arc Maps\GeoProcessing\models\models.gdb\Sakhalin_Well_Path_export'
export_df.spatial.to_featureclass(location=fc_location,has_z=True,has_m=False)
fc_route_location = fc_location + "s" #slap an s on the end
arcpy.lr.CreateRoutes(fc_location, "guid", fc_route_location,  "TWO_FIELDS", "top_md", "md", "UPPER_LEFT", 1, 0, "IGNORE", "INDEX")
arcpy.JoinField_management(fc_route_location,'guid',fc_location,'guid')
arcpy.DeleteField_management(fc_route_location,['Shape_Length_1','guid_1'])

<Result 'I:\\UTCS\\Region\\Russia\\Assets\\Sakhalin_1.Hou\\Regional\\Master\\SubSrfc\\Geo\\ArcGis\\Geol_Anal\\Data\\Arc Maps\\GeoProcessing\\models\\models.gdb\\Sakhalin_Well_Path_exports'>

In [19]:
SDE_well_path = r'I:\UTCS\Region\Russia\Assets\Sakhalin_1.Hou\Regional\Master\SubSrfc\Geo\ArcGis\Geol_Anal\Data\Arc Maps\GeoProcessing\models\SDE_SAKHALIN.sde\SDE_SAKHALIN.GIS.Sakhalin_Well_Paths'


arcpy.DeleteRows_management(SDE_well_path)
#I just copied the line below from ArcPro
arcpy.management.Append(fc_route_location,SDE_well_path, "NO_TEST", 'guid "guid" true true false 64 Text 0 0,First,#,Sakhalin_Well_Path_exports,guid,0,64;uwi "uwi" true true false 30 Text 0 0,First,#,Sakhalin_Well_Path_exports,uwi,0,30;boreholename "boreholename" true true false 64 Text 0 0,First,#,Sakhalin_Well_Path_exports,boreholename,0,64;insert_date "insert_date" true true false 8 Date 0 0,First,#,Sakhalin_Well_Path_exports,insert_date,-1,-1;md "md" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,md,-1,-1;tvd "tvd" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,tvd,-1,-1;folder_path "folder_path" true true false 200 Text 0 0,First,#,Sakhalin_Well_Path_exports,folder_path,0,200;field_name "field_name" true true false 24 Text 0 0,First,#,Sakhalin_Well_Path_exports,field_name,0,24;commingled_flag "commingled_flag" true true false 2 Text 0 0,First,#,Sakhalin_Well_Path_exports,commingled_flag,0,2;res_chayvo_14 "res_chayvo_14" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_chayvo_14,-1,-1;res_chayvo_16 "res_chayvo_16" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_chayvo_16,-1,-1;res_chayvo_17 "res_chayvo_17" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_chayvo_17,-1,-1;res_chayvo_18 "res_chayvo_18" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_chayvo_18,-1,-1;res_chayvo_19 "res_chayvo_19" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_chayvo_19,-1,-1;res_chayvo_20 "res_chayvo_20" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_chayvo_20,-1,-1;res_dagi_101 "res_dagi_101" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_dagi_101,-1,-1;res_dagi_102 "res_dagi_102" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_dagi_102,-1,-1;res_dagi_111 "res_dagi_111" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_dagi_111,-1,-1;res_dagi_121 "res_dagi_121" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_dagi_121,-1,-1;res_dagi_122 "res_dagi_122" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_dagi_122,-1,-1;res_dagi_131 "res_dagi_131" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_dagi_131,-1,-1;res_dagi_132 "res_dagi_132" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_dagi_132,-1,-1;res_dagi_133 "res_dagi_133" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_dagi_133,-1,-1;res_dagi_141 "res_dagi_141" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_dagi_141,-1,-1;res_dagi_142 "res_dagi_142" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_dagi_142,-1,-1;res_odoptu_211_leb_tsa "res_odoptu_211_leb_tsa" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_odoptu_211_leb_tsa,-1,-1;res_odoptu_211_od_tsa "res_odoptu_211_od_tsa" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_odoptu_211_od_tsa,-1,-1;res_odoptu_212 "res_odoptu_212" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_odoptu_212,-1,-1;res_odoptu_213_4_north "res_odoptu_213_4_north" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_odoptu_213_4_north,-1,-1;res_odoptu_221_north "res_odoptu_221_north" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_odoptu_221_north,-1,-1;res_odoptu_222 "res_odoptu_222" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_odoptu_222,-1,-1;res_odoptu_223 "res_odoptu_223" true true false 8 Double 8 38,First,#,Sakhalin_Well_Path_exports,res_odoptu_223,-1,-1;well_short_name "well_short_name" true true false 60 Text 0 0,First,#,Sakhalin_Well_Path_exports,well_short_name,0,60;folder "folder" true true false 60 Text 0 0,First,#,Sakhalin_Well_Path_exports,folder,0,60;top_md "top_md" true true false 4 Long 0 10,First,#,Sakhalin_Well_Path_exports,top_md,-1,-1', '', '')

<Result 'I:\\UTCS\\Region\\Russia\\Assets\\Sakhalin_1.Hou\\Regional\\Master\\SubSrfc\\Geo\\ArcGis\\Geol_Anal\\Data\\Arc Maps\\GeoProcessing\\models\\SDE_SAKHALIN.sde\\SDE_SAKHALIN.GIS.Sakhalin_Well_Paths'>

In [ ]:
feature_sdf.spatial.to_featurelayer('Sakhalin_well_paths',folder = 'Data Items')